In [11]:
import csv
import os
import re
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord


In [19]:
# ---------------------------
# 辅助函数：从 FASTA 记录头部解析转录本 ID
# 记录头的格式为: >{transcript_id}_{exon_number}(+/-)
# 这里使用正则表达式匹配“最后一个下划线”前的部分作为 transcript_id
def parse_transcript_id(header):
    # header可能形如 ">XM_004933515.5_1(+)"
    header = header.strip()
    if header.startswith(">"):
        header = header[1:]
    # 利用正则表达式，其中 (?P<transcript>.+) 贪婪匹配直到最后一个下划线，
    # 然后匹配 exon_number 和链号
  #  m = re.match(r"^(?P<transcript>.+)_(?P<exon>\d+)\(([+-])\)$", header)
    m = re.match(r"^(?P<transcript>.+)$", header)

    if m:
        return m.group("transcript")
    else:
        # 如果格式不符合预期，则简单地取第一个下划线前的部分
        return header.split("_")[0]


In [20]:
# ---------------------------
# 1. 解析同源组表格文件，构建同源组数据结构
orthogroups_file = "90percent_1_1_Lepidoptera_morethan50species_orthogroups.txt"  # 请替换为你的同源组表格文件路径

# 读取表格（假设为 tab 分隔），表头第一列为 Orthogroup，其余列为各物种，列名格式为 {species}.protein
with open(orthogroups_file, "r") as f:
    reader = csv.DictReader(f, delimiter="\t")
    # 获取所有物种列名（去除 Orthogroup 列）
    species_columns = [col for col in reader.fieldnames if col != "Orthogroup"]
    # 将所有行存入列表
    orthogroups = list(reader)


In [21]:
# ---------------------------
# 2. 对于每个物种，预先加载其外显子 FASTA 文件，建立字典： transcript_id -> list of SeqRecord
# FASTA 文件命名格式为 {species}.exon.fa，其中 species 为列名去掉 ".protein"
species_fasta = {}  # 键：species，值：字典 { transcript_id : [SeqRecord, ...] }
for col in species_columns:
    species = col.replace(".protein", "")
    fasta_file = f"{species}.exon.fasta"
    if not os.path.exists(fasta_file):
        print(f"警告：文件 {fasta_file} 不存在，跳过 {species}！")
        continue

    species_fasta[species] = {}
    # 解析 FASTA 文件中的每条记录
    for record in SeqIO.parse(fasta_file, "fasta"):
        tid = parse_transcript_id(record.description)
        if tid not in species_fasta[species]:
            species_fasta[species][tid] = []
        species_fasta[species][tid].append(record)
    print(f"加载 {fasta_file} 完成，共解析 {len(species_fasta[species])} 个转录本。")

加载 Bicyclus_anynana.exon.fasta 完成，共解析 1118 个转录本。
加载 Bombyx_mori.exon.fasta 完成，共解析 1225 个转录本。
加载 Danaus_plexippus.exon.fasta 完成，共解析 1155 个转录本。
加载 Heliconius_cydno.exon.fasta 完成，共解析 1287 个转录本。
加载 Heliconius_erato.exon.fasta 完成，共解析 1081 个转录本。
加载 Heliconius_melpomene.exon.fasta 完成，共解析 1122 个转录本。
加载 Helicoverpa_armigera.exon.fasta 完成，共解析 1117 个转录本。
加载 Junonia_coenia.exon.fasta 完成，共解析 1191 个转录本。
加载 Manduca_sexta.exon.fasta 完成，共解析 1174 个转录本。
加载 Ostrinia_furnacalis.exon.fasta 完成，共解析 1124 个转录本。
加载 Pieris_napi.exon.fasta 完成，共解析 1140 个转录本。
加载 Pieris_rapae.exon.fasta 完成，共解析 1136 个转录本。
加载 Plutella_xylostella.exon.fasta 完成，共解析 1137 个转录本。
加载 Spodoptera_frugiperda.exon.fasta 完成，共解析 1175 个转录本。
加载 Spodoptera_litura.exon.fasta 完成，共解析 1164 个转录本。
加载 Vanessa_cardui.exon.fasta 完成，共解析 1116 个转录本。


In [22]:
# ---------------------------
# 3. 对于每个同源组，提取各物种中对应转录本的所有外显子记录，
# 并在记录的转录本ID前加上物种名称，最后写入文件 {Orthogroup}.fasta
for row in orthogroups:
    orthogroup = row["Orthogroup"]
    records_to_write = []  # 存放所有待写入该同源组的 SeqRecord
    # 遍历各物种的列
    for col in species_columns:
        species = col.replace(".protein", "")
        transcript_ids_field = row[col].strip()
        # 如果该物种没有数据，则跳过
        if transcript_ids_field == "" or transcript_ids_field == "-":
            continue
        # 若单元格中有多个转录本ID，以逗号分隔
        transcript_ids = [tid.strip() for tid in transcript_ids_field.split(",")]
        for tid in transcript_ids:
            if species in species_fasta:
                if tid in species_fasta[species]:
                    # 对每条记录增加物种名称前缀
                    for record in species_fasta[species][tid]:
                        # 构造新的记录，新记录的 header 为 {species}_{原记录header}
                        new_record = SeqRecord(
                            seq=record.seq,
                            id=f"{species}_{record.id}",
                            description=f"{species}_{record.description}"
                        )
                        records_to_write.append(new_record)
                else:
                    print(f"警告：在 {species}.exon.fa 中未找到转录本 {tid}（同源组 {orthogroup}）。")
            else:
                print(f"警告：物种 {species} 的 FASTA 数据未加载。")
    # 写入输出文件，文件命名格式为 {Orthogroup}.fasta
    output_file = f"{orthogroup}.transcript.fasta"
    SeqIO.write(records_to_write, output_file, "fasta")
    print(f"同源组 {orthogroup} 写入 {output_file}，共 {len(records_to_write)} 条记录。")

同源组 OG0001729 写入 OG0001729.transcript.fasta，共 17 条记录。
同源组 OG0002314 写入 OG0002314.transcript.fasta，共 17 条记录。
同源组 OG0002514 写入 OG0002514.transcript.fasta，共 17 条记录。
同源组 OG0002565 写入 OG0002565.transcript.fasta，共 20 条记录。
同源组 OG0002567 写入 OG0002567.transcript.fasta，共 17 条记录。
同源组 OG0002614 写入 OG0002614.transcript.fasta，共 18 条记录。
同源组 OG0002826 写入 OG0002826.transcript.fasta，共 17 条记录。
同源组 OG0002833 写入 OG0002833.transcript.fasta，共 16 条记录。
同源组 OG0002883 写入 OG0002883.transcript.fasta，共 17 条记录。
同源组 OG0002939 写入 OG0002939.transcript.fasta，共 19 条记录。
同源组 OG0003078 写入 OG0003078.transcript.fasta，共 18 条记录。
同源组 OG0003182 写入 OG0003182.transcript.fasta，共 17 条记录。
同源组 OG0003193 写入 OG0003193.transcript.fasta，共 18 条记录。
同源组 OG0003259 写入 OG0003259.transcript.fasta，共 18 条记录。
同源组 OG0003438 写入 OG0003438.transcript.fasta，共 15 条记录。
同源组 OG0003533 写入 OG0003533.transcript.fasta，共 17 条记录。
同源组 OG0003594 写入 OG0003594.transcript.fasta，共 16 条记录。
同源组 OG0003599 写入 OG0003599.transcript.fasta，共 15 条记录。
同源组 OG0003722 写入 OG0003722.t

同源组 OG0008635 写入 OG0008635.transcript.fasta，共 17 条记录。
同源组 OG0008637 写入 OG0008637.transcript.fasta，共 17 条记录。
同源组 OG0008642 写入 OG0008642.transcript.fasta，共 15 条记录。
同源组 OG0008643 写入 OG0008643.transcript.fasta，共 16 条记录。
同源组 OG0008645 写入 OG0008645.transcript.fasta，共 16 条记录。
同源组 OG0008647 写入 OG0008647.transcript.fasta，共 15 条记录。
同源组 OG0008651 写入 OG0008651.transcript.fasta，共 17 条记录。
同源组 OG0008654 写入 OG0008654.transcript.fasta，共 16 条记录。
同源组 OG0008655 写入 OG0008655.transcript.fasta，共 17 条记录。
同源组 OG0008656 写入 OG0008656.transcript.fasta，共 17 条记录。
同源组 OG0008658 写入 OG0008658.transcript.fasta，共 17 条记录。
同源组 OG0008659 写入 OG0008659.transcript.fasta，共 15 条记录。
同源组 OG0008662 写入 OG0008662.transcript.fasta，共 15 条记录。
同源组 OG0008665 写入 OG0008665.transcript.fasta，共 16 条记录。
同源组 OG0008667 写入 OG0008667.transcript.fasta，共 16 条记录。
同源组 OG0008669 写入 OG0008669.transcript.fasta，共 17 条记录。
同源组 OG0008671 写入 OG0008671.transcript.fasta，共 16 条记录。
同源组 OG0008672 写入 OG0008672.transcript.fasta，共 17 条记录。
同源组 OG0008673 写入 OG0008673.t